In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install --upgrade scikit-learn

# Importing libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

# Reading files

In [ ]:
train_df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
train_df.head()

# Data Exploration

In [ ]:
ls=train_df[train_df['windspeed']==0]
ls.shape




In [ ]:
train_df.info()

## checking nulls

In [ ]:
train_df.isnull().sum()

**No null data**

## checking duplicates

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df.nunique()

In [ ]:
train_df.describe().T

In [ ]:
fig = plt.figure(figsize=(20,20))

for i,col in enumerate(list(train_df.columns)):
    plt.subplot(4,3,i+1)
    sns.histplot(train_df[col],kde=True)

## Features correlations

In [ ]:
corr_mtrx=train_df.corr()
corr_mtrx['count'].sort_values(ascending=False)

In [ ]:
numerical_attributes=['count','registered','casual','temp','atemp','season','windspeed','workingday','holiday','weather','humidity']

scatter_matrix(train_df[numerical_attributes],figsize=(18,12))
    

## Some insights

In [ ]:
train_df['season'].value_counts(ascending=True)

In [ ]:
train_df['holiday'].value_counts(ascending = False)

In [ ]:
train_df['workingday'].value_counts(ascending = False)

In [ ]:
train_df['weather'].value_counts(ascending = False)

In [ ]:
def visualize_categ(df):
    categ_col=['season','holiday','workingday','weather']
    for col in categ_col:
        sns.catplot(x=col , kind='count' ,data=df)
        plt.title(str(col))

In [ ]:
visualize_categ(train_df)

In [ ]:
def insights(df):
    for col in df:
        if col=='count':
            continue
        else:
            display(df[[col,'count']].groupby([col] , as_index=False).mean().sort_values(by='count',ascending = False).T)
    

In [ ]:
insights(train_df)

## New features


In [ ]:
train_df['datetime']=pd.to_datetime(train_df['datetime'])
train_df['year']=train_df['datetime'].dt.year
train_df['month']=train_df['datetime'].dt.month
train_df['day']=train_df['datetime'].dt.day
train_df.drop(columns=['datetime'],axis=1,inplace=True)

In [ ]:
train_df.info()

# Split data

In [ ]:
train_df_copy=train_df.copy()
label=train_df_copy['count']
train_df_copy.drop(columns=['count','casual','registered'] , axis=1,inplace=True)

In [ ]:
type(label)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_df_copy, label, train_size=0.8, test_size=0.2, random_state=0)


# Building models

## LinearRegression 

In [ ]:
model=LinearRegression()
model.fit(x_train, y_train)
pred=model.predict(x_valid)


In [ ]:
msle=mean_squared_log_error(abs(y_valid), abs(pred))
print(msle)

## RandomForest

In [ ]:
rf=RandomForestRegressor()
rf.fit(x_train, y_train)
rf_pred=rf.predict(x_valid)
rf_msle=mean_squared_log_error(y_valid, rf_pred)
rf_msle

# Testing

In [ ]:
test_df_copy=test_df.copy()
test_df_copy['datetime']=pd.to_datetime(test_df_copy['datetime'])
test_df_copy['year']=test_df_copy['datetime'].dt.year
test_df_copy['month']=test_df_copy['datetime'].dt.month
test_df_copy['day']=test_df_copy['datetime'].dt.day

test_df_copy.drop(columns=['datetime'],axis=1,inplace=True)

In [ ]:
print(test_df_copy.info())

In [ ]:
rf_pred=rf.predict(test_df_copy)


In [ ]:
d={'datetime':test_df['datetime'],'count':rf_pred}
ans=pd.DataFrame(d)
ans.to_csv('answer.csv',index=False)